### Real Estate Simulation
---

Simulate income of real-estate investment using
- Mortgage rate
- Price-to-rent ratio
- Multiple taxes
- Transaction fees
- Depreciations
- Simulated housing price growth

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from real_estate_sim import HousingMarket

%matplotlib inline
%load_ext autoreload
%autoreload 2
plt.style.use("ggplot")

In [8]:
HM = HousingMarket(growth=0.045, downpayment=0.2, 
                   price_to_rent=(11, 20), rental_tax_n_depreciation=0.2,
                   mkt_value=1e6)
results, df = HM.run_simulation()

Args provided:
                              values
mkt_value                  1000000.0
monthly_rent                    None
growth                         0.045
price_to_rent               (11, 20)
capital_gain                     0.2
transaction_fee                 0.05
mortgage_rate                   0.04
property_tax_annual            0.015
mortgage_fee                    0.01
downpayment                      0.2
management_fee                  0.08
rental_tax_n_depreciation        0.2
interest_rate                  0.015
capital                     200000.0


In [10]:
# annualized total return = appreciation + monthly income
results["total_ret"]

0.08115592011596018

In [11]:
results["appreciation"]

0.06452360862960793

In [12]:
# averaged approximated rental return per year
results["rental_ret"].head()

years
1    0.013181
2    0.013037
3    0.012914
4    0.012808
5    0.012715
dtype: float64

In [13]:
# income stream by year
results["income_stream"].head()

years
1    13499.682720
2    13953.998893
3    14444.338705
4    14969.774967
5    15529.512397
dtype: float64

In [14]:
df.head(20)

,month,years,growth,cum_growth,market_value,ptr_ratio,monthly_rent,monthly_mortgage,monthly_spending,net_income,discount_factor,discounted_net_income
0,1,1,0.003675,1.003675,1.003675e+06,11.000000,7603.597041,3857.515587,1357.504054,2388.577400,1.001250,2385.595405
1,2,1,0.003675,1.007363,1.007363e+06,11.025070,7614.185640,3857.515587,1360.468862,2396.201191,1.002502,2390.221902
2,3,1,0.003675,1.011065,1.011065e+06,11.050139,7624.828409,3857.515587,1363.448837,2403.863985,1.003755,2394.871984
3,4,1,0.003675,1.014780,1.014780e+06,11.075209,7635.525355,3857.515587,1366.443982,2411.565786,1.005009,2399.545543
4,5,1,0.003675,1.018510,1.018510e+06,11.100279,7646.276485,3857.515587,1369.454298,2419.306599,1.006266,2404.242471
5,6,1,0.003675,1.022252,1.022252e+06,11.125348,7657.081810,3857.515587,1372.479789,2427.086433,1.007523,2408.962659
6,7,1,0.003675,1.026009,1.026009e+06,11.150418,7667.941340,3857.515587,1375.520458,2434.905295,1.008783,2413.706002
7,8,1,0.003675,1.029779,1.029779e+06,11.175487,7678.855087,3857.515587,1378.576307,2442.763193,1.010044,2418.472396
8,9,1,0.003675,1.033564,1.033564e+06,11.200557,7689.823065,3857.515587,1381.647341,2450.660137,1.011306,2423.261736
9,10,1,0.003675,1.037362,1.037362e+06,11.225627,7700.845289,3857.515587,1384.733563,2458.596138,1.012571,2428.073920


In [15]:
df.tail(20)

,month,years,growth,cum_growth,market_value,ptr_ratio,monthly_rent,monthly_mortgage,monthly_spending,net_income,discount_factor,discounted_net_income
340,5,29,0.003675,3.493182,3.493182e+06,19.523677,14910.025839,3857.515587,3403.304117,7649.206134,1.531096,4995.902386
341,6,29,0.003675,3.506019,3.506019e+06,19.548747,14945.626211,3857.515587,3413.272222,7674.838402,1.533010,5006.385528
342,7,29,0.003675,3.518903,3.518903e+06,19.573816,14981.336224,3857.515587,3423.271025,7700.549611,1.534926,5016.886138
343,8,29,0.003675,3.531834,3.531834e+06,19.598886,15017.156194,3857.515587,3433.300617,7726.339990,1.536845,5027.404241
344,9,29,0.003675,3.544813,3.544813e+06,19.623955,15053.086438,3857.515587,3443.361085,7752.209765,1.538766,5037.939860
345,10,29,0.003675,3.557840,3.557840e+06,19.649025,15089.127274,3857.515587,3453.452519,7778.159168,1.540689,5048.493020
346,11,29,0.003675,3.570914,3.570914e+06,19.674095,15125.279023,3857.515587,3463.575009,7804.188427,1.542615,5059.063744
347,12,29,0.003675,3.584036,3.584036e+06,19.699164,15161.542006,3857.515587,57234.276030,-45930.249612,1.544543,-29737.104671
348,1,30,0.003675,3.597207,3.597207e+06,19.724234,15197.916543,3857.515587,3483.913514,7856.487441,1.546474,5080.257988
349,2,30,0.003675,3.610426,3.610426e+06,19.749304,15234.402958,3857.515587,3494.129711,7882.757660,1.548407,5090.881556


#### Analysis
---

The key to this analysis is to discover what conditions, real-estate investing becomes feasible or worth the of extra complexity it brings to life. While owning a house is arguably the most stable and accessible investment means, we need to keep a few factors in mind:

* Pros:
    * Income stream
    * Natural guard against emotions/irrational behaviors
    * Can be used personally
    * Easy access, and relatively low bar of research and entry
* Cons:
    * Deals with people - can be messy
    * Maintenance

We can first, model a few properties in known cities, like Pittsburgh, San Francisco, and San Ramon.

In city analysis, since we are investigating solely for the income stream purpose, we assume the properties are all purchased with 20% down-payment.

___

A few thoughts after playing with this env:

This simulation contains significant amount of assumption:
* future growth
    * number itself
    * linear increase
* price_to_rent changes
* accuracy of available downpayment and mortgage rate
* accuracy of the effective expense rates.

Due to historically low interest rate along with mortgage rate, real-estate investing appears to be a fantastic plan. Assuming the environment is accurate, a low downpayment can sufficiently generate annual appreciation of 10% without additional investment into the properties. However, because **most of the return comes in the form of market value appreciation**, we require additional research on the validity of growth assumption. If we purchase the property at peak of economic cycle, things might get incredibly ugly. If the investment aims for the monthly divident, return degrades to a much lower value, making me wonder it is better to invest in managed REITS or simply divident stocks. At least the latters require less of hussling.

I guess it is the best idea to hold back for now, and let the economy play out. 

#####  Pittsburgh

In [16]:
Pitt = HousingMarket(growth=0.035, downpayment=0.1, 
                   price_to_rent=(11, 20), rental_tax_n_depreciation=0.2,
                   mortgage_rate=0.025, mkt_value=5e5, interest_rate=0.001)
results, df = Pitt.run_simulation(years=30)

Args provided:
                             values
mkt_value                  500000.0
monthly_rent                   None
growth                        0.035
price_to_rent              (11, 20)
capital_gain                    0.2
transaction_fee                0.05
mortgage_rate                 0.025
property_tax_annual           0.015
mortgage_fee                   0.01
downpayment                     0.1
management_fee                 0.08
rental_tax_n_depreciation       0.2
interest_rate                 0.001
capital                     50000.0


In [17]:
print("total return: {:.4f}, appreciation return: {:.4f}, pct of pos income years: {:.4f}".format(results["total_ret"], results["appreciation"], (results["rental_ret"] > 0).mean()))

total return: 0.1039, appreciation return: 0.0891, pct of pos income years: 1.0000


A 7b3b multi-family house in Pittsburgh. The house has multiple rooms and was rented compartmentally. Valuation is around 550k and monthly rent is currently at 4200. 

In [18]:
print("annual income stream, in pct and in $")
print(results["rental_ret"].head())
print(results["income_stream"].head())

annual income stream, in pct and in $
years
1    0.015564
2    0.015008
3    0.014501
4    0.014037
5    0.013611
dtype: float64
years
1    7928.714743
2    7913.185956
3    7913.294641
4    7928.128971
5    7956.866549
dtype: float64


##### San Francisco


In [33]:
SF = HousingMarket(growth=0.03, downpayment=0.2, 
                   price_to_rent=(50, 50), rental_tax_n_depreciation=0.2,
                   mkt_value=1e6, property_tax=0.02)
results, df = SF.run_simulation(years=30)

Args provided:
                              values
mkt_value                  1000000.0
monthly_rent                    None
growth                          0.03
price_to_rent               (50, 50)
capital_gain                     0.2
transaction_fee                 0.05
mortgage_rate                   0.04
property_tax_annual             0.02
mortgage_fee                    0.01
downpayment                      0.2
management_fee                  0.08
rental_tax_n_depreciation        0.2
interest_rate                  0.015
capital                     200000.0
this lead to losing everything


/Users/George/Dropbox/investment-related/real-estate-invesment/real_estate_sim.py:121: RuntimeWarning: invalid value encountered in double_scalars
  # appreciation


In [34]:
print("total return: {:.4f}, appreciation return: {:.4f}, pct of pos income years: {:.4f}".format(results["total_ret"], results["appreciation"], (results["rental_ret"] > 0).mean()))
print("annual income stream, in pct and in $")
print(results["rental_ret"].head())
print(results["income_stream"].head())

total return: nan, appreciation return: 0.0409, pct of pos income years: 0.0000
annual income stream, in pct and in $
years
1   -0.042315
2   -0.041253
3   -0.040223
4   -0.039222
5   -0.038251
dtype: float64
years
1   -42999.206350
2   -43178.218052
3   -43362.600104
4   -43552.513618
5   -43748.124537
dtype: float64


Real estate investing in SF appears to be a losing business. The cumulative return is sub-zero.

##### San Ramon

In [35]:
SanRamon = HousingMarket(growth=0.03, downpayment=0.2, 
                   price_to_rent=(22, 22), rental_tax_n_depreciation=0.2,
                   mortgage_rate=0.03, mkt_value=1e6, interest_rate=0.02)
results, df = SanRamon.run_simulation(years=30)

Args provided:
                              values
mkt_value                  1000000.0
monthly_rent                    None
growth                          0.03
price_to_rent               (22, 22)
capital_gain                     0.2
transaction_fee                 0.05
mortgage_rate                   0.03
property_tax_annual            0.015
mortgage_fee                    0.01
downpayment                      0.2
management_fee                  0.08
rental_tax_n_depreciation        0.2
interest_rate                   0.02
capital                     200000.0


In [36]:
print("total return: {:.4f}, appreciation return: {:.4f}, pct of pos income years: {:.4f}".format(results["total_ret"], results["appreciation"], (results["rental_ret"] > 0).mean()))
print("annual income stream, in pct and in $")
print(results["rental_ret"].head())
print(results["income_stream"].head())

total return: 0.0401, appreciation return: 0.0357, pct of pos income years: 0.3000
annual income stream, in pct and in $
years
1   -0.014659
2   -0.013722
3   -0.012812
4   -0.011928
5   -0.011070
dtype: float64
years
1   -14896.292397
2   -14362.091718
3   -13811.865019
4   -13245.131519
5   -12661.396014
dtype: float64


In terms of San Ramon, the total return is 5.6% but most of it comes from appreciation.